In [7]:
import asyncio
from collections import OrderedDict
import logging
import os
from pathlib import Path
import json
import sys

import imageio.v3 as iio
from sklearn.cluster import HDBSCAN
from sklearn import metrics
from matplotlib import colormaps, colors
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from scipy.spatial.distance import euclidean
from scipy.spatial import distance_matrix
from scipy.stats import skew, skewtest, kurtosis, kurtosistest
import umap

from mime_db import MimeDb
#from pose_functions import *

MAX_PIXEL_MOVEMENT = 500 # Disregard movelets with > this many pixels movement (probably necessary)
MAX_3D_MOVEMENT = 10 # in meters/second

In [ ]:
# Build DB search indices for POEM embeddings and action embeddings
# Only needs to be run once (unless the data changes)
await db.assign_poem_embeddings([], reindex=True)
query = """
            CREATE INDEX ON pose
            USING ivfflat (ava_action vector_cosine_ops)
            ;
            """
await db._pool.execute(query)

In [8]:
def get_distribution_stats(distrib, plot=False):

    if len(distrib) == 0:
        return {"count": 0, "mean": 0, "median": 0, "stdev": 0, "skewness": 0, "kurtosis": 0}

    if skewtest(distrib).pvalue < .05:
        skewness = skew(distrib)
    else:
        skewness = 0

    if kurtosistest(distrib).pvalue < .05:
        kurtosis_value = kurtosis(distrib)
    else:
        kurtosis_value = 0

    if plot:
        plt.hist(distrib, bins="auto")  # arguments are passed to np.histogram
        plt.show()

    return {"count": len(distrib), "mean": np.mean(distrib), "median": np.median(distrib), "stdev": np.std(distrib), "skewness": skewness, "kurtosis": kurtosis_value}


def get_body_units(keypoints3d, camera):
    return get_body_unit(project_pose(keypoints3d, camera))


def get_body_unit(pose_proj):
    shoulders = euclidean(pose_proj[1], pose_proj[2])
    left_upper_arm = euclidean(pose_proj[1], pose_proj[3])
    right_upper_arm = euclidean(pose_proj[2], pose_proj[4])
    left_forearm = euclidean(pose_proj[3], pose_proj[5])
    right_forearm = euclidean(pose_proj[4], pose_proj[6])
    hips = euclidean(pose_proj[7], pose_proj[8])
    left_thigh = euclidean(pose_proj[7], pose_proj[9])
    right_thigh = euclidean(pose_proj[8], pose_proj[10])
    left_lower_leg = euclidean(pose_proj[9], pose_proj[11])
    right_lower_leg = euclidean(pose_proj[10], pose_proj[12])

    return np.mean([shoulders, left_upper_arm, right_upper_arm, left_forearm, right_forearm, hips, left_thigh, right_thigh, left_lower_leg, right_lower_leg])


def project_pose(keypoints3d, camera):
    kp_array = np.array(np.split(keypoints3d, 13))
    kp_camera = np.array(camera)

    return kp_array + kp_camera.T


def get_projected_pose_centroid(keypoints3d, camera):

    #proj_centroid = np.mean(kp_array, axis=0) + kp_camera.T
    proj_centroid = np.mean(project_pose(keypoints3d, camera), axis=0)

    proj_centroid[2] = proj_centroid[2]/200

    return proj_centroid


def get_projected_pose_bbox(keypoints3d, camera):
    ppose = project_pose(keypoints3d, camera)
    ppose[:,2] /= 200
    mins = np.min(ppose, axis=0).tolist()
    maxes = np.max(ppose, axis=0).tolist()
    
    return [mins, maxes]


def are_overlapping(bbox1, bbox2):
    # Each bbox is [[xmin, ymin, zmin], [xmax, ymax, zmax]]

    return ((bbox1[1][0] >= bbox2[0][0] and bbox1[1][0] <= bbox2[1][0]) \
            or (bbox1[0][0] <= bbox2[1][0] and bbox1[0][0] >= bbox2[0][0])) \
            and ((bbox1[1][1] >= bbox2[0][1] and bbox1[1][1] <= bbox2[1][1]) \
            or (bbox1[0][1] <= bbox2[1][1] and bbox1[0][1] >= bbox2[0][1])) \
            and ((bbox1[1][2] >= bbox2[0][2] and bbox1[1][2] <= bbox2[1][2]) \
            or (bbox1[0][2] <= bbox2[1][2] and bbox1[0][2] >= bbox2[0][2]))

    # From https://blender.stackexchange.com/questions/253355/collision-detection
    # isColliding = ((x_max >= x_min2 and x_max <= x_max2) \
    #                 or (x_min <= x_max2 and x_min >= x_min2)) \
    #                 and ((y_max >= y_min2 and y_max <= y_max2) \
    #                 or (y_min <= y_max2 and y_min >= y_min2)) \
    #                 and ((z_max >= z_min2 and z_max <= z_max2) \
    #                 or (z_min <= z_max2 and z_min >= z_min2))


In [ ]:
# Exploratory processin of a single video (for testing only, usually this cell is skipped)

VIDEO_FILE = "A_Letter_to_My_Nephew.mp4" # Just the name of the video file, no path

#db = await MimeDb.create()

video_path = Path("videos", VIDEO_FILE)

# Connect to the database

# Get video metadata
video_name = video_path.name
print(video_name)
video_id = await db.get_video_id(video_name)

print("VIDEO ID", video_id)

video_data = await db.get_video_by_id(video_id)
video_fps = video_data["fps"]
video_frame_count = video_data["frame_count"]

video_seconds = video_frame_count / video_fps

video_movelets = await db.get_movelet_data_from_video(video_id)
movelets_df = pd.DataFrame.from_records(video_movelets, columns=video_movelets[0].keys())

video_poses = await db.get_pose_data_from_video(video_id)

video_shots = await db.get_video_shots(video_id)
shots_df = pd.DataFrame.from_records(video_shots, columns=video_shots[0].keys())

poses_df = pd.DataFrame.from_records(video_poses, columns=video_poses[0].keys())


In [9]:
db = await MimeDb.create()

movelet_poem_embeds_by_video = []
global_3d_coco_13_by_video = []
action_embeds_by_video = []

async def get_video_stats(video_file):

    video_path = Path("videos", video_file)

    # Connect to the database

    # Get video metadata
    video_name = video_path.name
    print(video_name)
    video_id = await db.get_video_id(video_name)

    print("VIDEO ID", video_id)

    video_data = await db.get_video_by_id(video_id)
    video_fps = video_data["fps"]
    video_frame_count = video_data["frame_count"]

    video_seconds = video_frame_count / video_fps

    print("GETTING MOVELET DATA")

    video_movelets = await db.get_movelet_data_from_video(video_id)
    movelets_df = pd.DataFrame.from_records(video_movelets, columns=video_movelets[0].keys())

    print("GETTING POSE DATA")

    video_poses = await db.get_pose_data_from_video(video_id)

    print("GETTING SHOTS DATA")

    video_shots = await db.get_video_shots(video_id)
    shots_df = pd.DataFrame.from_records(video_shots, columns=video_shots[0].keys())

    poses_df = pd.DataFrame.from_records(video_poses, columns=video_poses[0].keys())

    video_3d_coco13_globals = np.array(poses_df["global3d_coco13"].to_list()) # DataFrame
    mean_3d_coco13_global = video_3d_coco13_globals.mean(axis=0).tolist() # ndarray
    median_3d_coco13_global = np.median(video_3d_coco13_globals, axis=0).tolist() # ndarray

    video_poem_embeddings = np.array(poses_df["poem_embedding"].to_list()) # DataFrame
    mean_poem_embedding = video_poem_embeddings.mean(axis=0).tolist() # ndarray
    median_poem_embedding = np.median(video_poem_embeddings, axis=0).tolist() # ndarray

    video_action_embeddings = np.array(poses_df["ava_action"].to_list()) # DataFrame
    mean_action_embedding = video_action_embeddings.mean(axis=0).tolist() # ndarray
    median_action_embedding = np.median(video_action_embeddings, axis=0).tolist() # ndarray

    print("TOTAL MOVELETS:", len(movelets_df))
    print("NON-MOTION MOVELETS:", len(movelets_df[movelets_df['movement'].isna()]))

    print("NON-3D-MOTION MOVELETS:", len(movelets_df[movelets_df['movement3d'].isna()]))

    print("MOVELETS WITH STILL MOTION:", len(movelets_df[movelets_df['movement'] == 0]))

    print("MOVELETS WITH STILL 3D MOTION:", len(movelets_df[movelets_df['movement3d'] == 0]))

    print("MOVELETS WITH MOVEMENT < 10px/sec:", len(movelets_df[(movelets_df['movement'] >= 0) & (movelets_df['movement'] < 10)]))

    print("MEAN MOVEMENT PER MOVELET (norm px/sec):", np.nanmean(movelets_df['movement']))
    print("MEDIAN MOVEMENT PER MOVELET (norm px/sec):", np.nanmedian(movelets_df['movement']))

    print("MEAN 3D MOVEMENT PER MOVELET (meters/sec):", np.nanmean(movelets_df['movement3d']))
    print("MEDIAN 3D MOVEMENT PER MOVELET (meters/sec):", np.nanmedian(movelets_df['movement3d']))

    nonnull_movelets_df = movelets_df.copy()
    nonnull_movelets_df['movement'] = nonnull_movelets_df['movement'].fillna(-1)

    movement_distribution = nonnull_movelets_df[nonnull_movelets_df['movement'] <= MAX_PIXEL_MOVEMENT]['movement']

    n, bins, patches = plt.hist(movement_distribution, bins=300)
    plt.xlabel("Movement (normalized pixels/sec)")
    plt.ylabel("# Movelets")
    top_bin = n[1:].argmax()
    print('most frequent movement bin: (' + str(bins[top_bin]) + ',' + str(bins[top_bin+1]) + ')')
    print('movement mode: '+ str((bins[top_bin] + bins[top_bin+1])/2))
    movement_mode = (bins[top_bin] + bins[top_bin+1])/2
    plt.show()

    movement_distribution_3d = nonnull_movelets_df[nonnull_movelets_df['movement3d'] <= MAX_3D_MOVEMENT]['movement3d']

    n, bins, patches = plt.hist(movement_distribution_3d, bins=300)
    plt.xlabel("3D Movement (meters/sec)")
    plt.ylabel("# Movelets")
    top_bin = n[1:].argmax()
    print('most frequent 3D movement bin: (' + str(bins[top_bin]) + ',' + str(bins[top_bin+1]) + ')')
    print('3D movement mode: '+ str((bins[top_bin] + bins[top_bin+1])/2))
    movement_mode_3d = (bins[top_bin] + bins[top_bin+1])/2
    plt.show()

    movelet_poem_embeds_by_video.append(nonnull_movelets_df.poem_embedding.to_list())

    global_3d_coco_13_by_video.append(video_3d_coco13_globals)

    action_embeds_by_video.append(video_action_embeddings)

    # Maybe use frozen_poses to compare pose distributions? Or just use all poses in the performance?
    #frozen_movelets = movelets_df[(movelets_df['movement'] >= 0) & (movelets_df['movement'] < movement_mode)].reset_index()
    #frozen_poses = frozen_movelets['norm'].tolist()

    poses_df = poses_df.merge(shots_df, left_on="frame", right_on="frame")

    poses_by_frame = poses_df.groupby("frame")

    all_distances = []
    all_mean_distances_per_frame = []
    poses_per_populated_frame = []
    overlaps_in_frame = []

    for frame, frame_poses_df in poses_by_frame:

        #print("FRAME", frame, "SHOT", frame_poses_df.iloc[0]["shot"], "POSES", len(frame_poses_df))
        
        poses_per_populated_frame.append(len(frame_poses_df))

        if len(frame_poses_df) <= 1:
            continue

        pose_centroids = frame_poses_df.apply(lambda x: get_projected_pose_centroid(x.keypoints3d, x.camera), axis=1)
        #print("POSE CENTROIDS")
        pose_centroids = [list(centroid) for centroid in pose_centroids]
        #print(pose_centroids)

        dist_matrix = distance_matrix(pose_centroids, pose_centroids)

        #print("DISTANCE MATRIX")
        #print(dist_matrix)

        upper_diagonal = list(dist_matrix[np.triu_indices(len(dist_matrix), k=1)])

        #print("UPPER DIAGONAL")
        #print(upper_diagonal)

        mean_interpose_distance = np.mean(upper_diagonal)

        all_distances.extend(upper_diagonal)
        all_mean_distances_per_frame.append(mean_interpose_distance)

        #if len(pose_centroids) > 1:
        #    print("EUCLIDEAN")
        #    print(euclidean(pose_centroids[0], pose_centroids[1]))

        pose_bboxes = frame_poses_df.apply(lambda x: get_projected_pose_bbox(x.keypoints3d, x.camera), axis=1)
        pose_bboxes = [list(bbox) for bbox in pose_bboxes]

        #print("POSE BOUNDING BOXES")
        #print(pose_bboxes)

        total_overlaps = 0
        for i, bbox1 in enumerate(pose_bboxes):
            for j, bbox2 in enumerate(pose_bboxes):
                if i >= j:
                    continue
                if are_overlapping(bbox1, bbox2):
                    #print("OVERLAP DETECTED IN FRAME", frame_poses_df.iloc[0]["frame"], "SHOT", frame_poses_df.iloc[0]["shot"], bbox1, bbox2)
                    total_overlaps += 1

        if total_overlaps > 1:
            overlaps_in_frame.append(total_overlaps)

        #pose_body_units = shot_poses_df.apply(lambda x: get_body_units(x.keypoints3d, x.camera), axis=1)
        #print("POSE BODY UNITS")
        #print(pose_body_units)

        #print(shot_poses_df["camera"]) # camera pos is relative to *each* detection
        #if shot > 101:
        #    break


    print("POSES PER POPULATED FRAME:")
    poses_ppf_stats = get_distribution_stats(poses_per_populated_frame, False)

    print("OVERLAPS (# POSES OVERLAPPING):")
    pose_overlaps_stats = get_distribution_stats(overlaps_in_frame, False)

    print("MEAN INTERPOSE DISTANCES PER POPULATED FRAME:")
    interpose_dist_ppf_stats = get_distribution_stats(all_mean_distances_per_frame, False)

    print("ALL INTERPOSE DISTANCES")
    interpose_dist_stats = get_distribution_stats(all_distances, False)


    # Merge non-null movelets with poses to calculate sidereal movement per pose per frame|second

    def get_sidereal_motion(movelet):

        # There shouldn't ever be more than one pose centroid that matches these queries (right???)
        start_centroid = poses_df[(poses_df["track_id"] == movelet["track_id"]) & (poses_df["frame"] == movelet["start_frame"])]["centroid_3d"].iloc[0]
        end_centroid = poses_df[(poses_df["track_id"] == movelet["track_id"]) & (poses_df["frame"] == movelet["end_frame"])]["centroid_3d"].iloc[0]
        
        return euclidean(start_centroid, end_centroid)

    movelets_df = nonnull_movelets_df[nonnull_movelets_df['movement'] <= MAX_PIXEL_MOVEMENT]

    poses_df["centroid_3d"] = poses_df.apply(lambda x: get_projected_pose_centroid(x.keypoints3d, x.camera), axis=1)

    movelets_df["sidereal"] = movelets_df.apply(get_sidereal_motion, axis=1)

    # Sidereal motion per second, based on movelet duration

    print("SIDEREAL MOTION PER MOVELET")
    sidereal_motion_stats = get_distribution_stats(movelets_df[movelets_df["sidereal"] < 1]["sidereal"].tolist(), False)

    poses_by_shot = poses_df.groupby("shot")

    tracks_per_shot = []

    for shot, shot_poses_df in poses_by_shot:

        total_tracks = shot_poses_df.track_id.unique()

        if len(total_tracks):
            tracks_per_shot.append(len(total_tracks))

        continue


    print("TRACKS PER SHOT:")
    tracks_per_shot_stats = get_distribution_stats(tracks_per_shot, False)

    print("NORMALIZED MOVEMENT STATS:")
    normalized_movement_stats = get_distribution_stats(movement_distribution, False)

    print("3D MOVEMENT STATS:")
    movement_3d_stats = get_distribution_stats(movement_distribution_3d, False)

    video_stats = {"seconds": video_seconds, "mean_poem_embedding": mean_poem_embedding, "median_poem_embedding": median_poem_embedding, "mean_3d_global": mean_3d_coco13_global, "median_3d_global": median_3d_coco13_global, "mean_action_embedding": mean_action_embedding, "median_action_embedding": median_action_embedding, "poses_ppf": poses_ppf_stats, "overlaps": pose_overlaps_stats, "interpose_dist_ppf": interpose_dist_ppf_stats, "interpose_dist": interpose_dist_stats, "sidereal": sidereal_motion_stats, "movement": normalized_movement_stats, "movement3d": movement_3d_stats, "tracks_per_shot": tracks_per_shot_stats}
    return video_stats

In [ ]:
# TO DO: Run extraction for all videos in the DB (or at least all 30 core videos)
# Write the derived statistics to a CSV(?) file
# Use the derived statistics to perform some kind of factor analysis
# Maybe a Bayesian classifier w/feature importance evaluation?

# For normalized 3D pose factor analysis
# Convert each global 3d pose into normalized vector of some size, maybe 25 x 25 x 25
# Maybe also try doing this with the Pr-VIPE 2.5D embeddings?
# Count the number of times each element appears in a pose
# Normalize all counts by the max in all elements

In [ ]:
all_videos = await db.get_available_videos()
videos_df = pd.DataFrame.from_records(all_videos, columns=all_videos[0].keys())
all_video_stats = {}

video_names = [video_name for video_name in videos_df.video_name.tolist() if not video_name.startswith("Don_Giovanni-")]

# Recalculate all video stats
#for video_name in video_names:
#   all_video_stats[video_name] = await get_video_stats(video_name)

print(video_names)

import pickle
movelet_poem_embeds_by_video = pickle.load(open("stylometry/poem_embeds_by_video.pkl", "rb"))
global_3d_coco_13_by_video = pickle.load(open("stylometry/global_3d_coco_13_by_video.pkl", "rb"))
action_embeds_by_video = pickle.load(open("stylometry/action_embeds_by_video.pkl", "rb"))

# Load cached stats
all_video_stats = json.load(open("stylometry/video_stats.json", "r", encoding="utf-8"))

# Get stats for the final video(s) on the list
#for v in range(30, len(video_names)):
#    all_video_stats[video_names[v]] = await get_video_stats(video_names[v])

In [ ]:
# Augment video data with directors and save stats to files

import os
import pickle

director_videos = {}
video_directors = {}
director_videos["BillTJones"] = []
director_videos["Castellucci"] = []
director_videos["Warlikowski"] = []
director_videos["Other"] = []
director_ids = {"BillTJones": 0, "Castellucci": 1, "Warlikowski": 2, "Other": 3}

video_slugs = [
"JLN",
"JAD",
"JQP",
"JDM",
"JFD",
"JHD",
"JPP",
"JSP",
"JST",
"JWS",
"CDF",
"CDA",
"CDG",
"CGD",
"CIN",
"CPW",
"CPU",
"CRM",
"CRG",
"CMF",
"WBC",
"WDG",
"WES",
"WIT",
"WM1",
"WM2",
"WMC",
"WCH",
"WTF",
"WWB",
"JAE",
]

for i, video_name in enumerate(video_names):
    if i < 10 or i == 30:
        director_videos["BillTJones"].append(video_name)
        video_directors[video_name] = director_ids["BillTJones"]
    elif i >= 10 and i < 20:
        director_videos["Castellucci"].append(video_name)
        video_directors[video_name] = director_ids["Castellucci"]
    elif i >= 20 and i < 30:
        director_videos["Warlikowski"].append(video_name)
        video_directors[video_name] = director_ids["Warlikowski"]
    else:
        director_videos["Other"].append(video_name)
        video_directors[video_name] = director_ids["Other"]


if not os.path.isdir("stylometry"):
    os.mkdir("stylometry")

for video_name in all_video_stats:
    if video_name.startswith("Don_Giovanni-"):
        continue

    all_video_stats[video_name]["director"] = video_directors[video_name]

json.dump(all_video_stats, open("stylometry/video_stats.json", "w", encoding="utf-8"), indent=4)

#pickle.dump(movelet_poem_embeds_by_video, open("stylometry/poem_embeds_by_video.pkl", "wb"))
#pickle.dump(global_3d_coco_13_by_video, open("stylometry/global_3d_coco_13_by_video.pkl", "wb"))
#pickle.dump(action_embeds_by_video, open("stylometry/action_embeds_by_video.pkl", "wb"))

#movelet_poem_embeds_by_video = pickle.load(open("stylometry/poem_embeds_by_video.pkl", "rb"))
#global_3d_coco_13_by_video = pickle.load(open("stylometry/global_3d_coco_13_by_video.pkl", "rb"))
#action_embeds_by_video = pickle.load(open("stylometry/action_embeds_by_video.pkl", "rb"))


In [32]:
# Group poem and action embeddings and 3D global poses by director for embedding analysis

poem_embeddings_by_director = {}
action_embeddings_by_director = {}
global_3d_coco_13_by_director = {}

poem_embeddings_by_director["BillTJones"] = []
poem_embeddings_by_director["Castellucci"] = []
poem_embeddings_by_director["Warlikowski"] = []
for poem_embeds in movelet_poem_embeds_by_video[:10]:
    poem_embeddings_by_director["BillTJones"].extend(poem_embeds)
for poem_embeds in movelet_poem_embeds_by_video[10:20]:
    poem_embeddings_by_director["Castellucci"].extend(poem_embeds)
for poem_embeds in movelet_poem_embeds_by_video[20:30]:
    poem_embeddings_by_director["Warlikowski"].extend(poem_embeds)
poem_embeddings_by_director["BillTJones"].extend(movelet_poem_embeds_by_video[30])

billtjones_mean_poem_embedding = np.mean(poem_embeddings_by_director["BillTJones"], axis=0)
castellucci_mean_poem_embedding = np.mean(poem_embeddings_by_director["Castellucci"], axis=0)
warlikowski_mean_poem_embedding = np.mean(poem_embeddings_by_director["Warlikowski"], axis=0)

billtjones_median_poem_embedding = np.median(poem_embeddings_by_director["BillTJones"], axis=0)
castellucci_median_poem_embedding = np.median(poem_embeddings_by_director["Castellucci"], axis=0)
warlikowski_median_poem_embedding = np.median(poem_embeddings_by_director["Warlikowski"], axis=0)

action_embeddings_by_director["BillTJones"] = []
action_embeddings_by_director["Castellucci"] = []
action_embeddings_by_director["Warlikowski"] = []
for action_embeds in action_embeds_by_video[:10]:
    action_embeddings_by_director["BillTJones"].extend(action_embeds)
for action_embeds in action_embeds_by_video[10:20]:
    action_embeddings_by_director["Castellucci"].extend(action_embeds)
for action_embeds in action_embeds_by_video[20:30]:
    action_embeddings_by_director["Warlikowski"].extend(action_embeds)
action_embeddings_by_director["BillTJones"].extend(action_embeds_by_video[30])

billtjones_mean_action_embedding = np.mean(action_embeddings_by_director["BillTJones"], axis=0)
castellucci_mean_action_embedding = np.mean(action_embeddings_by_director["Castellucci"], axis=0)
warlikowski_mean_action_embedding = np.mean(action_embeddings_by_director["Warlikowski"], axis=0)

billtjones_median_action_embedding = np.median(action_embeddings_by_director["BillTJones"], axis=0)
castellucci_median_action_embedding = np.median(action_embeddings_by_director["Castellucci"], axis=0)
warlikowski_median_action_embedding = np.median(action_embeddings_by_director["Warlikowski"], axis=0)

global_3d_coco_13_by_director["BillTJones"] = []
global_3d_coco_13_by_director["Castellucci"] = []
global_3d_coco_13_by_director["Warlikowski"] = []
for global_3d_coco_13 in global_3d_coco_13_by_video[:10]:
    global_3d_coco_13_by_director["BillTJones"].extend(global_3d_coco_13)
for global_3d_coco_13 in global_3d_coco_13_by_video[10:20]:
    global_3d_coco_13_by_director["Castellucci"].extend(global_3d_coco_13)
for global_3d_coco_13 in global_3d_coco_13_by_video[20:30]:
    global_3d_coco_13_by_director["Warlikowski"].extend(global_3d_coco_13)
global_3d_coco_13_by_director["BillTJones"].extend(global_3d_coco_13_by_video[30])

billtjones_mean_global_3d_coco_13 = np.mean(global_3d_coco_13_by_director["BillTJones"], axis=0)
castellucci_mean_global_3d_coco_13 = np.mean(global_3d_coco_13_by_director["Castellucci"], axis=0)
warlikowski_mean_global_3d_coco_13 = np.mean(global_3d_coco_13_by_director["Warlikowski"], axis=0)

billtjones_median_global_3d_coco_13 = np.median(global_3d_coco_13_by_director["BillTJones"], axis=0)
castellucci_median_global_3d_coco_13 = np.median(global_3d_coco_13_by_director["Castellucci"], axis=0)
warlikowski_median_global_3d_coco_13 = np.median(global_3d_coco_13_by_director["Warlikowski"], axis=0)

In [34]:
# Convert video motion, distance summary statistics into feature vectors
# Also extract per-video average embeddings for classification test
# And write the feature summary stats to a TSV file

video_labels = []
video_features = []

video_mean_poem_embeddings = []
video_median_poem_embeddings = []
video_mean_action_embeddings = []
video_median_action_embeddings = []
video_mean_global_3d_coco_13 = []
video_median_global_3d_coco_13 = []

summary_metrics_to_skip = ["overlaps", "seconds", "director", "mean_3d_global", "mean_poem_embedding", "median_poem_embedding", "mean_action_embedding", "median_3d_global", "median_action_embedding"]
metrics_to_skip = ["seconds", "poses_ppf", "tracks_per_shot", "interpose_dist", "director", "mean_3d_global", "mean_poem_embedding", "median_poem_embedding", "mean_action_embedding", "median_3d_global", "median_action_embedding"]
subkeys_to_skip = ["count", "skewness", "kurtosis"]
slugs_to_skip = ["overlaps_stdev", "overlaps_kurtosis", "overlaps_skewness"]

feature_names = []

summary_file = open("stylometry/video_summary_stats.tsv", "w")
summary_file.write("video\tdirector")

for i, video_name in enumerate(all_video_stats):

    if video_name.startswith("Don_Giovanni-"):
        continue

    video_labels.append(all_video_stats[video_name]["director"])
    video_mean_poem_embeddings.append(all_video_stats[video_name]["mean_poem_embedding"])
    video_median_poem_embeddings.append(all_video_stats[video_name]["median_poem_embedding"])
    video_mean_action_embeddings.append(all_video_stats[video_name]["mean_action_embedding"])
    video_median_action_embeddings.append(all_video_stats[video_name]["median_action_embedding"])
    video_mean_global_3d_coco_13.append(all_video_stats[video_name]["mean_3d_global"])
    video_median_global_3d_coco_13.append(all_video_stats[video_name]["median_3d_global"])

    # First build the feature vec for the numpy array
    feature_vec = []
    for metric in all_video_stats[video_name]:
        if metric in metrics_to_skip:
            continue
        for subkey in all_video_stats[video_name][metric]:
            if subkey in subkeys_to_skip:
                continue
            feature_slug = metric + "_" + subkey
            if feature_slug in slugs_to_skip:
                continue
            if i == 0:
                feature_names.append(feature_slug)

            feature_vec.append(all_video_stats[video_name][metric][subkey])
    video_features.append(feature_vec)

    # Then build the list of summary stats for the TSV file
    summary_vec = []
    for metric in all_video_stats[video_name]:
        if metric in summary_metrics_to_skip:
            continue
        for subkey in all_video_stats[video_name][metric]:
            if subkey in subkeys_to_skip:
                continue
            feature_slug = metric + "_" + subkey
            if feature_slug in slugs_to_skip:
                continue
            if i == 0:
                summary_file.write(f"\t{feature_slug}")

            summary_vec.append(all_video_stats[video_name][metric][subkey])

    if i == 0:
        summary_file.write("\n") 
    summary_stats = [video_name, all_video_stats[video_name]['director']] + summary_vec
    summary_file.write("\t".join([str(stat) for stat in summary_stats]) + "\n")

summary_file.close()

video_feat_matrix = np.array(video_features)

label_colormap = {0: "red", 1: "green", 2: "blue", 3: "orange"}
label_colors = [label_colormap[l] for l in video_labels]

In [ ]:
# "Dumbest" possible classifier, using "leave one out" approach based on video features

from sklearn.metrics.pairwise import cosine_similarity

hits = 0
misses = 0

predictions = []

for i, video_name in enumerate(video_names[:31]):

    # XXX Note that this overwrites some values from the cells above
    # (but just run those cells again to repopulate the values)
    video_features_by_director = {}

    video_features_by_director["BillTJones"] = []
    video_features_by_director["Castellucci"] = []
    video_features_by_director["Warlikowski"] = []

    for e, _ in enumerate(video_names[:31]):
        if e == i:
            continue
        if e < 10 or e == 30:
            video_features_by_director["BillTJones"].append(video_features[e])
        elif e >= 10 and e < 20:
            video_features_by_director["Castellucci"].append(video_features[e])
        else:
            video_features_by_director["Warlikowski"].append(video_features[e])

    billtjones_mean_video_features = np.mean(video_features_by_director["BillTJones"], axis=0)
    castellucci_mean_video_features = np.mean(video_features_by_director["Castellucci"], axis=0)
    warlikowski_mean_video_features = np.mean(video_features_by_director["Warlikowski"], axis=0)

    directors_matrix = np.stack((billtjones_mean_video_features, castellucci_mean_video_features, warlikowski_mean_video_features), axis=0)

    test_video_features = video_features[i]
    true_label = video_labels[i]

    sims = cosine_similarity(directors_matrix, np.array([test_video_features]))

    pred_label = np.argmax(sims)
    print(video_name, "Prediction:", pred_label, "True:", true_label)

    predictions.append(pred_label)

    if pred_label == true_label:
        hits += 1
    else:
        misses += 1

print("Hits:", hits, "Misses:", misses, f"{hits*100/(hits+misses):.2f}%")

confusion_matrix = metrics.confusion_matrix(video_labels[:31], predictions)
#cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["BillTJones", "Castellucci", "Warlikowski"])
cm_display = metrics.ConfusionMatrixDisplay.from_predictions(video_labels, predictions, display_labels=["BillTJones", "Castellucci", "Warlikowski"], colorbar=False)
cm_display.plot()
plt.show()

In [ ]:
# "Dumbest" possible classifier, using "leave one out" approach based on POEM embeddings

from sklearn.metrics.pairwise import cosine_similarity

hits = 0
misses = 0

predictions = []

for i, video_name in enumerate(video_names):

    # XXX Note that this overwrites some values from the cells above
    # (but just run those cells again to repopulate the values)
    poem_embeddings_by_director = {}

    poem_embeddings_by_director["BillTJones"] = []
    poem_embeddings_by_director["Castellucci"] = []
    poem_embeddings_by_director["Warlikowski"] = []

    for e, _ in enumerate(video_names[:31]):
        if e == i:
            continue
        if e < 10 or e == 30:
            poem_embeddings_by_director["BillTJones"].extend(movelet_poem_embeds_by_video[e])
        elif e >= 10 and e < 20:
            poem_embeddings_by_director["Castellucci"].extend(movelet_poem_embeds_by_video[e])
        else:
            poem_embeddings_by_director["Warlikowski"].extend(movelet_poem_embeds_by_video[e])

    billtjones_poem_embedding = np.mean(poem_embeddings_by_director["BillTJones"], axis=0)
    castellucci_poem_embedding = np.mean(poem_embeddings_by_director["Castellucci"], axis=0)
    warlikowski_poem_embedding = np.mean(poem_embeddings_by_director["Warlikowski"], axis=0)

    directors_matrix = np.stack((billtjones_poem_embedding, castellucci_poem_embedding, warlikowski_poem_embedding), axis=0)

    avg_video_embedding = video_mean_poem_embeddings[i]
    true_label = video_labels[i]

    sims = cosine_similarity(directors_matrix, np.array([avg_video_embedding]))

    pred_label = np.argmax(sims)
    print(video_name, "Prediction:", pred_label, "True:", true_label)

    predictions.append(pred_label)

    if pred_label == true_label:
        hits += 1
    else:
        misses += 1

print("Hits:", hits, "Misses:", misses, f"{hits*100/(hits+misses):.2f}%")

confusion_matrix = metrics.confusion_matrix(video_labels, predictions)
#cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["BillTJones", "Castellucci", "Warlikowski"])
cm_display = metrics.ConfusionMatrixDisplay.from_predictions(video_labels, predictions, display_labels=["BillTJones", "Castellucci", "Warlikowski"], colorbar=False)
cm_display.plot()
plt.show()

In [ ]:
# "Dumbest" possible classifier, using "leave one out" approach based on AVA embeddings

from sklearn.metrics.pairwise import cosine_similarity

hits = 0
misses = 0

predictions = []

for i, video_name in enumerate(video_names):

    # XXX Note that this overwrites some values from the cells above
    # (but just run those cells again to repopulate the values)
    action_embeddings_by_director = {}

    action_embeddings_by_director["BillTJones"] = []
    action_embeddings_by_director["Castellucci"] = []
    action_embeddings_by_director["Warlikowski"] = []

    for e, _ in enumerate(video_names):
        if e == i:
            continue
        if e < 10 or e == 30:
            action_embeddings_by_director["BillTJones"].extend(action_embeds_by_video[e])
        elif e >= 10 and e < 20:
            action_embeddings_by_director["Castellucci"].extend(action_embeds_by_video[e])
        else:
            action_embeddings_by_director["Warlikowski"].extend(action_embeds_by_video[e])

    billtjones_action_embedding = np.mean(action_embeddings_by_director["BillTJones"], axis=0)
    castellucci_action_embedding = np.mean(action_embeddings_by_director["Castellucci"], axis=0)
    warlikowski_action_embedding = np.mean(action_embeddings_by_director["Warlikowski"], axis=0)

    directors_matrix = np.stack((billtjones_action_embedding, castellucci_action_embedding, warlikowski_action_embedding), axis=0)

    avg_video_embedding = video_mean_action_embeddings[i]
    true_label = video_labels[i]

    sims = cosine_similarity(directors_matrix, np.array([avg_video_embedding]))

    pred_label = np.argmax(sims)
    print(video_name, "Prediction:", pred_label, "True:", true_label)

    predictions.append(pred_label)

    if pred_label == true_label:
        hits += 1
    else:
        misses += 1

print("Hits:", hits, "Misses:", misses, f"{hits*100/(hits+misses):.2f}%")

confusion_matrix = metrics.confusion_matrix(video_labels, predictions)
#cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["BillTJones", "Castellucci", "Warlikowski"])
cm_display = metrics.ConfusionMatrixDisplay.from_predictions(video_labels, predictions, display_labels=["BillTJones", "Castellucci", "Warlikowski"], colorbar=False)
cm_display.plot()
plt.show()

In [ ]:
# "Dumbest" possible classifier, using "leave one out" approach based on 3d global poses

from sklearn.metrics.pairwise import cosine_similarity

hits = 0
misses = 0

predictions = []

for i, video_name in enumerate(video_names):

    # XXX Note that this overwrites some values from the cells above
    # (but just run those cells again to repopulate the values)
    global_3d_coco_13_by_director = {}

    global_3d_coco_13_by_director["BillTJones"] = []
    global_3d_coco_13_by_director["Castellucci"] = []
    global_3d_coco_13_by_director["Warlikowski"] = []

    for e, _ in enumerate(video_names):
        if e == i:
            continue
        if e < 10 or e == 30:
            global_3d_coco_13_by_director["BillTJones"].extend(global_3d_coco_13_by_video[e])
        elif e >= 10 and e < 20:
            global_3d_coco_13_by_director["Castellucci"].extend(global_3d_coco_13_by_video[e])
        else:
            global_3d_coco_13_by_director["Warlikowski"].extend(global_3d_coco_13_by_video[e])

    billtjones_gobal_3d_coco_13 = np.mean(global_3d_coco_13_by_director["BillTJones"], axis=0)
    castellucci_global_3d_coco_13 = np.mean(global_3d_coco_13_by_director["Castellucci"], axis=0)
    warlikowski_global_3d_coco_13 = np.mean(global_3d_coco_13_by_director["Warlikowski"], axis=0)

    directors_matrix = np.stack((billtjones_gobal_3d_coco_13, castellucci_global_3d_coco_13, warlikowski_global_3d_coco_13), axis=0)

    avg_video_global_3d_coco_13 = video_mean_global_3d_coco_13[i]
    true_label = video_labels[i]

    sims = cosine_similarity(directors_matrix, np.array([avg_video_global_3d_coco_13]))

    pred_label = np.argmax(sims)
    print(video_name, "Prediction:", pred_label, "True:", true_label)

    predictions.append(pred_label)

    if pred_label == true_label:
        hits += 1
    else:
        misses += 1

print("Hits:", hits, "Misses:", misses, f"{hits*100/(hits+misses):.2f}%")

confusion_matrix = metrics.confusion_matrix(video_labels, predictions)
#cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["BillTJones", "Castellucci", "Warlikowski"])
cm_display = metrics.ConfusionMatrixDisplay.from_predictions(video_labels, predictions, display_labels=["BillTJones", "Castellucci", "Warlikowski"], colorbar=False)
cm_display.plot()
plt.show()

In [ ]:
# Basic 3D plot of pose coordinates

COCO_13_SKELETON = [
    (12, 10),
    (10, 8),
    (13, 11),
    (11, 9),
    (8, 9),
    (2, 8),
    (3, 9),
    (2, 3),
    (2, 4),
    (3, 5),
    (4, 6),
    (5, 7),
    (1, 2),
    (1, 3),
]

COCO_COLORS = [
    "orangered",
    "orange",
    "blue",
    "lightblue",
    "darkgreen",
    "red",
    "lightgreen",
    "pink",
    "plum",
    "purple",
    "brown",
    "saddlebrown",
    "mediumorchid",
    "gray",
    "salmon",
    "chartreuse",
    "lightgray",
    "darkturquoise",
    "goldenrod",
]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

#pose_3d = np.array(billtjones_gobal_3d_coco_13).reshape(13,3)
#pose_3d = np.array(castellucci_global_3d_coco_13).reshape(13,3)
pose_3d = np.array(warlikowski_global_3d_coco_13).reshape(13,3)


ax.scatter(pose_3d[:,0], pose_3d[:,1], pose_3d[:,2])

for i, seg in enumerate(COCO_13_SKELETON):
    #line_color = ImageColor.getrgb(COCO_COLORS[i])
    plt.plot([pose_3d[seg[0] - 1][0], pose_3d[seg[1] - 1][0]], [pose_3d[seg[0] - 1][1], pose_3d[seg[1] - 1][1]], [pose_3d[seg[0] - 1][2], pose_3d[seg[1] - 1][2]], 'b-')

plt.show()

In [ ]:
# Basic visualization of the avg values of various movement/distance stats by video/director

for i, ftn in enumerate(feature_names):
    print(ftn)
    #print(video_feat_matrix[i, :])
    plt.scatter(video_labels, video_feat_matrix[:, i], c=label_colors)
    plt.xticks([0, 1, 2], ["BillTJones", "Castellucci", "Warlikowski"], rotation=45)
    ax = plt.gca()
    for j, video_name in enumerate(all_video_stats):
        ax.annotate(video_name, (video_labels[j], video_feat_matrix[j, i]))
    plt.show()

In [ ]:
# Train a classifier on all but one; see where that one is classified,
# repeat for all videos (leave-one-out).
# Potentially repeat the process for different random seeds, if the
# classification algorithm is nondeterministic

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
#import xgboost as xgb
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neural_network import MLPClassifier

N_SEEDS = 1

hits = 0
misses = 0
predicted = []
actual = []

#X = video_feat_matrix
X = video_mean_poem_embeddings
#X = video_mean_action_embeddings
#X = video_mean_global_3d_coco_13

for r in range(N_SEEDS):
    for h, held_out_vector in enumerate(X):

        X_train = np.delete(X, h, 0)
        y_train = np.delete(video_labels, h, 0)
        #X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=r)

        #clf = MLPClassifier()
        #clf = NuSVC(random_state=42)
        clf = GaussianNB()
        #clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
        #clf = RandomForestClassifier(random_state=r)
        clf.fit(X_train, y_train)
        #clf.fit(X_train,  y_train, eval_set=[(X_test, y_test)])

        #pred = clf.predict(held_out_vector.reshape(1, -1))[0]
        pred = clf.predict([held_out_vector])[0]

        if pred == video_labels[h]:
            hits += 1
        else:
            misses += 1

        actual.append(video_labels[h])
        predicted.append(pred)

        #print("For ", video_names[h], "pred is", pred, "true is", video_labels[h])

print(f"Hits: {hits}, Misses: {misses}, {hits*100/(hits+misses):.2f}%")

confusion_matrix = metrics.confusion_matrix(actual, predicted)
cm_display = metrics.ConfusionMatrixDisplay.from_predictions(actual, predicted, display_labels=["BillTJones", "Castellucci", "Warlikowski"], colorbar=False)
plt.title("GNB: AVA")
cm_display.plot()
#plt.show()

In [ ]:
from sklearn.model_selection import cross_validate, cross_val_score, KFold
import random
from datetime import datetime

# Another way to do it
# scoring = ['precision_macro', 'recall_macro']
# clf = RandomForestClassifier(random_state=0)
# scores = cross_validate(clf, video_feat_matrix, video_labels, scoring=scoring)
# print("precision scores:", scores["test_precision_macro"], "recall scores:", scores["test_recall_macro"])

N_SEEDS = 10

print("10-fold cross validation with 10 random seeds for video feature vectors")
all_scores = []
for i in range(N_SEEDS):
    r = random.seed(datetime.now().timestamp())
    clf = RandomForestClassifier(random_state=r)
    #clf = GaussianNB()
    cv = KFold(n_splits=10, random_state=r)
    scores = cross_val_score(
        clf, video_feat_matrix, video_labels, scoring='accuracy', cv=cv, n_jobs=-1)
    all_scores.extend(scores)
print('Accuracy: %.3f ,\nStandard Deviations :%.3f' %
    (np.mean(all_scores), np.std(all_scores)))

print("10-fold cross validation with 10 random seeds for POEM vectors")
for i in range(N_SEEDS):
    r = random.seed(datetime.now().timestamp())
    clf = RandomForestClassifier(random_state=r)
    #clf = GaussianNB()
    cv = KFold(n_splits=10, random_state=r)
    scores = cross_val_score(
        clf, video_mean_poem_embeddings, video_labels, scoring='accuracy', cv=cv, n_jobs=-1)
    all_scores.extend(scores)
print('Accuracy: %.3f ,\nStandard Deviations :%.3f' %
    (np.mean(all_scores), np.std(all_scores)))

print("10-fold cross validation with 10 random seeds for action vectors")
for i in range(N_SEEDS):
    r = random.seed(datetime.now().timestamp())
    clf = RandomForestClassifier(random_state=r)
    #clf = GaussianNB()
    cv = KFold(n_splits=10, random_state=r)
    scores = cross_val_score(
        clf, video_mean_action_embeddings, video_labels, scoring='accuracy', cv=cv, n_jobs=-1)
    all_scores.extend(scores)
print('Accuracy: %.3f ,\nStandard Deviations :%.3f' %
    (np.mean(all_scores), np.std(all_scores)))

print("10-fold cross validation with 10 random seeds for global 3D poses")
for i in range(N_SEEDS):
    r = random.seed(datetime.now().timestamp())
    clf = RandomForestClassifier(random_state=r)
    #clf = GaussianNB()
    cv = KFold(n_splits=10, random_state=r)
    scores = cross_val_score(
        clf, video_mean_global_3d_coco_13, video_labels, scoring='accuracy', cv=cv, n_jobs=-1)
    all_scores.extend(scores)
print('Accuracy: %.3f ,\nStandard Deviations :%.3f' %
    (np.mean(all_scores), np.std(all_scores)))

In [ ]:
from sklearn.inspection import permutation_importance
import time

#X_train, X_test, y_train, y_test = train_test_split(video_mean_poem_embeddings, video_labels, test_size=0.33, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(video_feat_matrix, video_labels, test_size=0.33, random_state=42)#
X_train, X_test, y_train, y_test = train_test_split(video_mean_global_3d_coco_13, video_labels, test_size=0.33, random_state=42)

clf = RandomForestClassifier(random_state=0)
#clf = GaussianNB()
#clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)

clf.fit(X_train, y_train)
#clf.fit(X_train,  y_train, eval_set=[(X_test, y_test)])

start_time = time.time()
perm_importance = permutation_importance(
    clf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

vector_names = [f"feature {i}" for i in range(len(video_mean_poem_embeddings[0]))]

coco_13_points = ["nose",  # 1
    "left_shoulder",  # 2
    "right_shoulder",  # 3
    "left_elbow",  # 4
    "right_elbow",  # 5
    "left_wrist",  # 6
    "right_wrist",  # 7
    "left_hip",  # 8
    "right_hip",  # 9
    "left_knee",  # 10
    "right_knee",  # 11
    "left_ankle",  # 12
    "right_ankle",  # 13
]

keypoint_names = [[f"{item}X", f"{item}Y", f"{item}Z"] for item in coco_13_points]
keypoint_names = sum(keypoint_names, [])

#clf_importances = pd.Series(perm_importance.importances_mean, index=vector_names)
#clf_importances = pd.Series(perm_importance.importances_mean, index=feature_names)
clf_importances = pd.Series(perm_importance.importances_mean, index=keypoint_names)

fig, ax = plt.subplots()
clf_importances.plot.bar(yerr=perm_importance.importances_std, ax=ax)
ax.set_title("Feature importances (permutation): ")
ax.set_ylabel("Mean accuracy decrease")
ax.tick_params(axis='x', labelrotation=40)
plt.setp(ax.xaxis.get_majorticklabels(), ha='right')
fig.tight_layout()
plt.show()

sorted_idx = perm_importance.importances_mean.argsort()
#plt.barh(np.array(vector_names)[sorted_idx], perm_importance.importances_mean[sorted_idx], xerr=perm_importance.importances_std[sorted_idx])
#plt.barh(np.array(feature_names)[sorted_idx], perm_importance.importances_mean[sorted_idx], xerr=perm_importance.importances_std[sorted_idx])
plt.barh(np.array(keypoint_names)[sorted_idx], perm_importance.importances_mean[sorted_idx], xerr=perm_importance.importances_std[sorted_idx])
plt.xlabel("Permutation Importance")
plt.show()

In [ ]:
# Find the most similar poses to a video's (or director's) average embedding
# (can either search only that video, or all of them)

pose_subquery = "TRUE"

query = f"""
            SELECT pose.video_id, video.video_name, pose.frame, pose.pose_idx, pose.bbox, pose.norm, pose.keypoints, poem_embedding <=> '{billtjones_mean_poem_embedding.tolist()}' AS distance, frame.shot AS shot FROM pose, frame, video
            WHERE {pose_subquery} AND video.id = pose.video_id AND frame.video_id = pose.video_id AND pose.frame = frame.frame ORDER BY distance
            LIMIT 500
        """

similar_poses = await db._pool.fetch(query)


In [ ]:
# Plot/draw most similar poses to the average embedding

import imageio.v3 as iio
from lib.pose_drawing import pad_and_excerpt_image, draw_armatures
from PIL import Image, ImageDraw

UPSCALE = 5

async def get_frame_and_highlight_pose(video_id, frame, keypoints, bbox):
    
    keypoints = np.array([round(kp) for kp in keypoints]).reshape((13, 3))
    xmin = int(np.rint(np.min(keypoints[:, 0])))
    ymin = int(np.rint(np.min(keypoints[:, 1])))
    xmax = int(np.rint(np.max(keypoints[:, 0])))
    ymax = int(np.rint(np.max(keypoints[:, 1])))

    video = await db.get_video_by_id(video_id)
    video_path = f"/videos/{video['video_name']}"
    frame_img = iio.imread(video_path, index=frame - 1, plugin="pyav")

    frame_img = Image.fromarray(np.uint8(frame_img)).convert('RGB')

    frame_img = frame_img.resize((frame_img.width * UPSCALE, frame_img.height * UPSCALE))

    overlay_img = ImageDraw.Draw(frame_img)

    overlay_img = draw_armatures(keypoints, overlay_img, None)

    overlay_img.rectangle([(xmin * UPSCALE, ymin * UPSCALE), (xmax * UPSCALE, ymax * UPSCALE)], outline ="purple", width = 4 * UPSCALE) 

    return frame_img


async def get_cropped_pose_on_image(video_id, frame, keypoints):

    keypoints = np.array([round(kp) for kp in keypoints]).reshape((13, 3))
    xmin = int(np.rint(np.min(keypoints[:, 0])))
    ymin = int(np.rint(np.min(keypoints[:, 1])))
    xmax = int(np.rint(np.max(keypoints[:, 0])))
    ymax = int(np.rint(np.max(keypoints[:, 1])))
    width = xmax - xmin
    height = ymax - ymin

    video = await db.get_video_by_id(video_id)
    video_path = f"/videos/{video['video_name']}"
    frame_img = iio.imread(video_path, index=frame - 1, plugin="pyav")

    excerpted_img = pad_and_excerpt_image(frame_img, xmin, ymin, width, height)

    excerpted_img = Image.fromarray(np.uint8(excerpted_img)).convert('RGB')

    excerpted_img = excerpted_img.resize((width * UPSCALE, height * UPSCALE))

    overlay_img = ImageDraw.Draw(excerpted_img)

    overlay_img = draw_armatures(keypoints, overlay_img, None, xmin, ymin)

    return excerpted_img

for pose in similar_poses[150:200]:

    pose_image = await get_cropped_pose_on_image(pose.get("video_id"), pose.get("frame"), pose.get("keypoints"))
    frame_image = await get_frame_and_highlight_pose(pose.get("video_id"), pose.get("frame"), pose.get("keypoints"), pose.get("bbox"))

    print(pose.get("video_name"), "frame", pose.get("frame"), "pose", pose.get("pose_idx"))
    fig, ax = plt.subplots()
    plt.axis('off')
    plt.imshow(frame_image)
    plt.show()
    plt.axis('off')
    plt.imshow(pose_image)
    plt.show()
    #display(example_image)

None

In [ ]:
# Find the most similar actions to a video's (or director's) average embedding
# (can either search only that video, or all of them)

pose_subquery = "TRUE"

query = f"""
            SELECT pose.video_id, video.video_name, pose.frame, pose.pose_idx, pose.bbox, pose.norm, pose.keypoints, action_labels, ava_action <=> '{billtjones_mean_action_embedding.tolist()}' AS distance, frame.shot AS shot FROM pose, frame, video
            WHERE {pose_subquery} AND video.id = pose.video_id AND frame.video_id = pose.video_id AND pose.frame = frame.frame ORDER BY distance
            LIMIT 500
        """

similar_actions = await db._pool.fetch(query)

In [ ]:
for action_pose in similar_actions[:20]:
    print(action_pose.get("video_name"), "frame", action_pose.get("frame"), "pose", action_pose.get("pose_idx"))
    print(action_pose.get("action_labels"))

    frame_image = await get_frame_and_highlight_pose(action_pose.get("video_id"), action_pose.get("frame"), action_pose.get("keypoints"), action_pose.get("bbox"))
    pose_image = await get_cropped_pose_on_image(action_pose.get("video_id"), action_pose.get("frame"), action_pose.get("keypoints"))

    fig, ax = plt.subplots()
    plt.axis('off')
    plt.imshow(frame_image)
    plt.show()
    plt.axis('off')
    plt.imshow(pose_image)
    plt.show()

In [ ]:
# Project large numbers of embeddings into 2D, attempt to colorize by director
# and highlight where directors' (and specific videos'?) avg embeddings are.
# NOTE: Running on the full embeddings set would exceed the RAM of most (maybe all)
# machines, so we sample by SAMPLE_RATE.
# Note also that there are almost no identical embeddings, and their dimensionality
# is such that rounding the values and removing duplicates only decreases their size
# by a tiny fraction.

import matplotlib.patches as mpatches

SAMPLE_RATE = 100 # 1/100 poses will be used

# For use with the Tensorflow Embedding Projector or similar visualizations
poem_metadata = open("stylometry/sampled_pose_metadata.tsv", "w")
poem_metadata.write("Director\tWork\n")

all_poem_embeddings = []
embedding_label_colors = []

# XXX Gets messy due to the first 10 Bill T. Jones videos being indices 0-9, and the last one
# being at the end of the list (index 30). Need to make sure metadata aligns with sampled embeddings

e = 0
for w in [*range(10), 30]:
    dir_name = "BillTJones"
    video_name = video_names[w]
    work_embeds = movelet_poem_embeds_by_video[w]
    for embed in work_embeds:
        if e % SAMPLE_RATE == 0:
            poem_metadata.write(f"{dir_name}\t{video_name}\n")
        e += 1
for w in range(10, 20):
    dir_name = "Castellucci"
    video_name = video_names[w]
    work_embeds = movelet_poem_embeds_by_video[w]
    for embed in work_embeds:
        if e % SAMPLE_RATE == 0:
            poem_metadata.write(f"{dir_name}\t{video_name}\n")
        e += 1
for w in range(20, 30):
    dir_name = "Warlikowski"
    video_name = video_names[w]
    work_embeds = movelet_poem_embeds_by_video[w]
    for embed in work_embeds:
        if e % SAMPLE_RATE == 0:
            poem_metadata.write(f"{dir_name}\t{video_name}\n")
        e += 1

# for w, work_embeds in enumerate(movelet_poem_embeds_by_video):
#     video_name = video_names[w]
#     if w < 10 or w == 30:
#         dir_name = "BillTJones"
#     elif w < 20:
#         dir_name = "Castellucci"
#     else:
#         dir_name = "Warlikowski"
#     for embed in work_embeds:
#         if e % SAMPLE_RATE == 0:
#             poem_metadata.write(f"{dir_name}\t{video_name}\n")
#         e += 1

i = 0
for emb in poem_embeddings_by_director["BillTJones"]:
    if i % SAMPLE_RATE == 0:
        all_poem_embeddings.append(emb)
        embedding_label_colors.append(label_colormap[0])
    i +=1
for emb in poem_embeddings_by_director["Castellucci"]:
    if i % SAMPLE_RATE == 0:
        all_poem_embeddings.append(emb)
        embedding_label_colors.append(label_colormap[1])
    i += 1
for emb in poem_embeddings_by_director["Warlikowski"]:
    if i % SAMPLE_RATE == 0:
        all_poem_embeddings.append(emb)
        embedding_label_colors.append(label_colormap[2])
    i += 1

all_poem_embeddings.extend(video_mean_poem_embeddings)

for i, video_name in enumerate(video_names):
    if i < 10 or i == 30:
        dir_name = "BillTJones"
    elif i < 20:
        dir_name = "Castellucci"
    else:
        dir_name = "Warlikowski"
    poem_metadata.write(f"{dir_name}\t{video_name}_AVG\n")

video_colors = ["red"] * 10
video_colors.extend(["green"] * 10)
video_colors.extend(["blue"] * 10)
video_colors.extend(["red"])

all_poem_embeddings.extend([billtjones_mean_poem_embedding, castellucci_mean_poem_embedding, warlikowski_mean_poem_embedding])
#all_poem_embeddings.extend([billtjones_median_poem_embedding, castellucci_median_poem_embedding, warlikowski_median_poem_embedding])

poem_metadata.write("BillTJones\tDirector_Average\n")
poem_metadata.write("Castellucci\tDirector_Average\n")
poem_metadata.write("Warlikowski\tDirector_Average\n")

print("CALCULATING UMAP PROJECTION ON", len(all_poem_embeddings), "POEM embeddings")

clusterable_embedding = umap.UMAP(
    metric="cosine",
    n_neighbors=25,
    min_dist=0,
    n_components=2,
    n_jobs=8,
).fit_transform(all_poem_embeddings)

plt.figure(figsize=(10,10))

# XXX This gets messy...
# all but the last 34 embeddings are the individual sampled pose embeddings for each video,
# followed by the average embedding for each video (n=31) and for each director (n=3)
plt.scatter(clusterable_embedding[:-34, 0], clusterable_embedding[:-34, 1], s=2, alpha=0.3, c=embedding_label_colors)
plt.scatter(clusterable_embedding[-34:-3, 0], clusterable_embedding[-34:-3, 1], s=50, alpha=0.8, marker="D", c=video_colors, edgecolors=["black"]* 31)
plt.scatter(clusterable_embedding[-3:, 0], clusterable_embedding[-3:, 1], s=200, marker="H", c=["red", "green", "blue"], edgecolors=["black", "black", "black"])

# ax = plt.gca()
# for s in range(30):
#     ax.annotate(video_slugs[s], (clusterable_embedding[-33 + s, 0], clusterable_embedding[-33 + s, 1]))

red_patch = mpatches.Patch(color='red', label='BillTJones')
green_patch = mpatches.Patch(color='green', label='Castellucci')
blue_patch = mpatches.Patch(color='blue', label='Warlikowski')
plt.legend(handles=[red_patch, green_patch, blue_patch])

plt.show()

poem_metadata.close()

with open("stylometry/sampled_pose_embeddings.tsv", "w") as poem_data:
    for emb in all_poem_embeddings:
        poem_data.write("\t".join([str(e) for e in emb]))
        poem_data.write("\n")

In [ ]:
# Project large numbers of embeddings into 2D, attempt to colorize by director
# and highlight where directors' (and specific videos'?) avg embeddings are.
# NOTE: Running on the full embeddings set would exceed the RAM of most (maybe all)
# machines, so we sample by SAMPLE_RATE.
# Note also that there are almost no identical embeddings, and their dimensionality
# is such that rounding the values and removing duplicates only decreases their size
# by a tiny fraction.

import matplotlib.patches as mpatches

SAMPLE_RATE = 100 # 1/100 poses will be used

all_action_embeddings = []
embedding_label_colors = []

for i, emb in enumerate(action_embeddings_by_director["BillTJones"]):
    if i % SAMPLE_RATE == 0:
        all_action_embeddings.append(emb)
        embedding_label_colors.append(label_colormap[0])
for i, emb in enumerate(action_embeddings_by_director["Castellucci"]):
    if i % SAMPLE_RATE == 0:
        all_action_embeddings.append(emb)
        embedding_label_colors.append(label_colormap[1])
for i, emb in enumerate(action_embeddings_by_director["Warlikowski"]):
    if i % SAMPLE_RATE == 0:
        all_action_embeddings.append(emb)
        embedding_label_colors.append(label_colormap[2])

all_action_embeddings.extend([billtjones_mean_action_embedding, castellucci_mean_action_embedding, warlikowski_mean_action_embedding])
#all_action_embeddings.extend([billtjones_median_action_embedding, castellucci_median_action_embedding, warlikowski_median_action_embedding])

print("CALCULATING UMAP PROJECTION ON", len(all_action_embeddings), "ACTION embeddings")

clusterable_embedding = umap.UMAP(
    metric="cosine",
    n_neighbors=15,
    min_dist=0.0,
    n_components=2,
    n_jobs=8,
).fit_transform(all_action_embeddings)

plt.figure(figsize=(10,10))

plt.scatter(clusterable_embedding[:-3, 0], clusterable_embedding[:-3, 1], s=2, alpha=0.3, c=embedding_label_colors)
#plt.scatter(clusterable_embedding[-6:-3, 0], clusterable_embedding[-6:-3, 1], s=200, marker="s", c=["red", "green", "blue"], edgecolors=["white", "white", "white"])
plt.scatter(clusterable_embedding[-3:, 0], clusterable_embedding[-3:, 1], s=200, marker="h", c=["red", "green", "blue"], edgecolors=["white", "white", "white"])

red_patch = mpatches.Patch(color='red', label='BillTJones')
green_patch = mpatches.Patch(color='green', label='Castellucci')
blue_patch = mpatches.Patch(color='blue', label='Warlikowski')

plt.legend(handles=[red_patch, green_patch, blue_patch])

plt.show()

- Z distances seem to be exaggerated -- apparently by 200:1? See PHALP phalp/utils/utils.py
- Focal length is 5000 (pixels?) -- not sure how to use this

Based on body part ratios (i.e., considering average of all shoulder widths, waist widths, upper arm lengths and lower arm lengths) the projected 3D pose coordinate units appear to be in meters.

## Metrics of interest for director-to-video clustering analysis

- Difference of action recognition embedding vectors. How best to compare? Just cosine distance of averaged embeddings? Some kind of multi-dimensional KL divergence? Would be nice to be able to take into account the normalized histogram of values in each of the 62 elements of the action recognition vectors.

- Difference of "held" 3D pose vectors/representations. How to compare? Maybe just 3D Euclidean distance between averaged normalized poses? As flattened embeddings, they'd be 100 x 100 x 100 = 1,000,000-element vectors (curse of dimensionality) unless of course they're downsampled further to like 10 x 10

## Motion:

- Normalized: per-pixel movement within a pose's frame of reference, for each movelet -- already calculated
- Sidereal (relative to the "scenery"): normalized per-meter (?) movement for each movelet -- need to join movelets with poses (again)

## Closeness of figures in frames/shots:

One primary shortcoming of this is that we usually can't see the whole stage, and reconstructing the entire scene based on multiple camera views (if those are provided at all) is beyond current technology, though various NeRF/Gaussian Splatting methods are getting closer. So we're mostly just stuck with what we get in each shot, which makes this analysis highly "contaminated" by the cinematography/cinematic style. Only way to sidestep this for now is to note that we're including multiple films for each stage director, so hopefully the influence of the filmic cinematography of any given recording will be attenuated/smoothed over when they've compared in "bulk."

- For each populated frame in the film, with the denominator = # of frames with actual people detected in them. Ignore unpopulated frames; incorporate the frame rate to get all of these metrics per second:
  - Get the dist of number of people per populated frame/sec
- For the rest of these, only consider frames with multiple people:
  - Get the distance dist between people per populated frame/sec (see below for calculations)
  - Get the number of overlapping people per populated frame/sec (see below for calculations)

### Secondary analyses

- Maybe try to count and distribution-ize the number of distinct people in a shot -- this is essentially the number of tracks in a shot (tracking is done by appearance, including costumes though without much weight given to faces). Could help to cover the case of people coming into and out of a shot. Though it may just further confuse the issue, and in any case this should be very sensitive to shot length -- so may need to normalize by shot length somehow for the distribution to be useful.

### Alternative shot-based analysis (probably not necessary)

- For each shot:

  - For the first (?) frame in the shot:

    - For each pose in the frame:

      - Get its position as the median of x, y, z/200 (?) coords
      - Also get its 3D bounding box if doing collision detection

    - Compute Euclidean distance matrix of all of the pose centroids, put this into a distribution, extract statistics
      - Determine how many (if any) 3D bounding boxes intersect (touching) []
